# Model Training

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import os
import sys
from database.queries import get_all_users
from dataframe_builder import build_user_dataframe

In [ ]:
df = build_user_dataframe(1) #training data set
# df.head(5)

,session_id,training_day,date,body_weight,mood,dchest,arms,waist,legs,shoulders,training_duration_minutes,sleep_hours,stress_level,program_phase,exercise,muscle_group,weight,reps,volume
0,1,FULLBODY1,2012-01-02,70.1,tired,82.8,25.2,60.4,45.7,98.6,71.5,6.4,3,deload,Back Squat,legs,39.4,9,354.6
1,1,FULLBODY1,2012-01-02,70.1,tired,82.8,25.2,60.4,45.7,98.6,71.5,6.4,3,deload,Barbell Bench Press,chest,39.9,8,319.2
2,1,FULLBODY1,2012-01-02,70.1,tired,82.8,25.2,60.4,45.7,98.6,71.5,6.4,3,deload,Lat Pulldown,back,35.6,10,356.0
3,1,FULLBODY1,2012-01-02,70.1,tired,82.8,25.2,60.4,45.7,98.6,71.5,6.4,3,deload,Romanian Deadlift,hamstrings,37.1,6,222.6
4,1,FULLBODY1,2012-01-02,70.1,tired,82.8,25.2,60.4,45.7,98.6,71.5,6.4,3,deload,Assisted Dip,triceps,20.1,11,221.1


In [5]:
chest_df = df[df['muscle_group'] == 'chest']
chest_df

,session_id,training_day,date,body_weight,mood,dchest,arms,waist,legs,shoulders,training_duration_minutes,sleep_hours,stress_level,program_phase,exercise,muscle_group,weight,reps,volume
1,1,FULLBODY1,2012-01-02,70.1,tired,82.8,25.2,60.4,45.7,98.6,71.5,6.4,3,deload,Barbell Bench Press,chest,39.9,8,319.2
11,2,FULLBODY2,2012-01-03,70.0,tired,86.1,24.6,59.7,44.8,97.6,60.2,5.8,4,hypertrophy,Cable Flye,chest,19.1,11,210.1
15,3,FULLBODY3,2012-01-06,70.2,okay,84.9,24.3,59.4,45.7,99.0,58.1,7.2,1,hypertrophy,Dumbbell Incline Press,chest,30.1,12,361.2
22,4,FULLBODY1,2012-01-09,70.3,okay,85.4,24.0,60.9,45.6,98.4,55.1,7.8,1,hypertrophy,Barbell Bench Press,chest,47.5,7,332.5
32,5,FULLBODY2,2012-01-11,70.2,tired,85.0,24.4,60.5,44.0,98.9,53.2,6.9,3,hypertrophy,Cable Flye,chest,32.5,7,227.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13966,1996,FULLBODY1,2024-09-30,110.7,okay,132.8,38.0,95.7,70.6,154.2,59.0,7.4,3,hypertrophy,Barbell Bench Press,chest,1357.5,7,9502.5
13976,1997,FULLBODY2,2024-10-01,110.7,tired,131.7,39.1,94.2,72.8,155.9,58.6,5.0,5,hypertrophy,Cable Flye,chest,977.5,9,8797.5
13980,1998,FULLBODY3,2024-10-04,110.6,okay,133.3,38.6,92.4,73.2,154.7,64.9,7.0,2,hypertrophy,Dumbbell Incline Press,chest,1175.0,12,14100.0
13987,1999,FULLBODY1,2024-10-07,110.6,okay,134.8,38.9,94.0,73.7,154.2,63.2,7.2,6,hypertrophy,Barbell Bench Press,chest,1190.0,8,9520.0


In [ ]:
chest_df = chest_df.drop(columns =['session_id', 'date', 'training_day','weight', 'reps'], axis = 1)

In [8]:
chest_df

,body_weight,mood,dchest,arms,waist,legs,shoulders,training_duration_minutes,sleep_hours,stress_level,program_phase,exercise,muscle_group,weight,reps,volume
1,70.1,tired,82.8,25.2,60.4,45.7,98.6,71.5,6.4,3,deload,Barbell Bench Press,chest,39.9,8,319.2
11,70.0,tired,86.1,24.6,59.7,44.8,97.6,60.2,5.8,4,hypertrophy,Cable Flye,chest,19.1,11,210.1
15,70.2,okay,84.9,24.3,59.4,45.7,99.0,58.1,7.2,1,hypertrophy,Dumbbell Incline Press,chest,30.1,12,361.2
22,70.3,okay,85.4,24.0,60.9,45.6,98.4,55.1,7.8,1,hypertrophy,Barbell Bench Press,chest,47.5,7,332.5
32,70.2,tired,85.0,24.4,60.5,44.0,98.9,53.2,6.9,3,hypertrophy,Cable Flye,chest,32.5,7,227.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13966,110.7,okay,132.8,38.0,95.7,70.6,154.2,59.0,7.4,3,hypertrophy,Barbell Bench Press,chest,1357.5,7,9502.5
13976,110.7,tired,131.7,39.1,94.2,72.8,155.9,58.6,5.0,5,hypertrophy,Cable Flye,chest,977.5,9,8797.5
13980,110.6,okay,133.3,38.6,92.4,73.2,154.7,64.9,7.0,2,hypertrophy,Dumbbell Incline Press,chest,1175.0,12,14100.0
13987,110.6,okay,134.8,38.9,94.0,73.7,154.2,63.2,7.2,6,hypertrophy,Barbell Bench Press,chest,1190.0,8,9520.0


In [9]:
# X_train = chest_df[]
y_train = chest_df['volume']
y_train.shape

(2000,)

In [2]:
while(True):
    print('s')

s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s


: 